# Accessing ITS_LIVE data hosted by AWS
- this is a notebook from Scott H

## Which AWS data center hosts the data?

**AWS us-east-1**

check this at: 

In [1]:
!nslookup https://its-live-data.s3.amazonaws.com

Server:		128.117.64.45
Address:	128.117.64.45#53

Non-authoritative answer:
https://its-live-data.s3.amazonaws.com	canonical name = s3-1-w.amazonaws.com.
s3-1-w.amazonaws.com	canonical name = s3-w.us-east-1.amazonaws.com.
Name:	s3-w.us-east-1.amazonaws.com
Address: 52.217.102.116



## Checking if data available over certain regions

In [2]:
import geopandas as gpd

In [3]:
gf_all = gpd.read_file('https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json')


In [4]:
gf_all.head(4)

,fill-opacity,fill,roi_percent_coverage,data_epsg,geometry_epsg,datacube_exist,zarr_url,geometry
0,0.984866,red,1.513354,EPSG:32718,"{'type': 'Polygon', 'coordinates': [[[400000, ...",1,http://its-live-data.s3.amazonaws.com/datacube...,"POLYGON ((-76.41134 -50.54338, -75.00000 -50.5..."
1,0.956741,red,4.325908,EPSG:32718,"{'type': 'Polygon', 'coordinates': [[[400000, ...",1,http://its-live-data.s3.amazonaws.com/datacube...,"POLYGON ((-76.38517 -49.64426, -75.00000 -49.6..."
2,0.941388,red,5.861168,EPSG:32718,"{'type': 'Polygon', 'coordinates': [[[500000, ...",1,http://its-live-data.s3.amazonaws.com/datacube...,"POLYGON ((-75.00000 -54.14810, -73.46930 -54.1..."
3,0.881647,red,11.835322,EPSG:32718,"{'type': 'Polygon', 'coordinates': [[[500000, ...",1,http://its-live-data.s3.amazonaws.com/datacube...,"POLYGON ((-75.00000 -52.35029, -73.53212 -52.3..."


In [5]:
type(gf_all)

geopandas.geodataframe.GeoDataFrame

In [6]:
gf_all.explore()

In [7]:


# geopandas crop to himat
xmin = 70
xmax = 110
ymin = 25
ymax = 50
gf = gf_all.cx[xmin:xmax, ymin:ymax]
len(gf)

383

In [8]:
gf.explore()

In [9]:
url = gf.iloc[0].zarr_url
url

'http://its-live-data.s3.amazonaws.com/datacubes/v02/N30E060/ITS_LIVE_vel_EPSG32642_G0120_X550000_Y3850000.zarr'

In [10]:
s3obj = url.replace('.s3.amazonaws.com', '').replace('http://', 's3://')
s3obj

's3://its-live-data/datacubes/v02/N30E060/ITS_LIVE_vel_EPSG32642_G0120_X550000_Y3850000.zarr'

## Do all of these s3 zarr objects exist? How much space do they take up? 

**having trouble downloading aws cli** sent email to help desk about getting permissions

In [11]:
!aws --no-sign-request s3 ls {s3obj}/

/bin/bash: aws: command not found


In [17]:
import s3fs
fs  = s3fs.S3FileSystem(anon=True)
fs

In [12]:
#make the s3 url a col in the gdf
gf['zarr_s3'] = gf.zarr_url.str.replace('.s3.amazonaws.com','').str.replace('http://','s3://')


/var/folders/y9/k06gyzdx25z64l_tdt_q2xv9s680sz/T/ipykernel_7485/3620601657.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  gf['zarr_s3'] = gf.zarr_url.str.replace('.s3.amazonaws.com','').str.replace('http://','s3://')
/Users/emarshall/miniconda3/envs/mynewbook/lib/python3.10/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [13]:
#also save the index as a column (tile id)
gf['tile_id'] = gf.index

/Users/emarshall/miniconda3/envs/mynewbook/lib/python3.10/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [14]:
gf.iloc[0]

fill-opacity                                                     0.752848
fill                                                                  red
roi_percent_coverage                                            24.715192
data_epsg                                                      EPSG:32642
geometry_epsg           {'type': 'Polygon', 'coordinates': [[[500000, ...
datacube_exist                                                          1
zarr_url                http://its-live-data.s3.amazonaws.com/datacube...
geometry                POLYGON ((69 34.341303, 70.087144 34.336476, 7...
zarr_s3                 s3://its-live-data/datacubes/v02/N30E060/ITS_L...
tile_id                                                               309
Name: 309, dtype: object

In [15]:
# 'datacube_exist' is a col in gf
# unique() returns unique values of series object
gf.datacube_exist.unique()

array([1])

`lexists` is a method to check whetehr a given path exists or not. supply: `os.path.lexists(path)` that you can apply to the fs object (type = s3fs.core.S3FileSystem)

In [18]:

gf['path_exists'] = gf.zarr_s3.apply(lambda x: fs.lexists(x))

In [19]:
gf.path_exists.unique()


array([ True])

Use `groupby` to check which of the supplied zarr urls for hma exist:

In [20]:
gf.groupby('path_exists')['zarr_url'].count()

path_exists
True    383
Name: zarr_url, dtype: int64

### Which s3 paths don't exist?

In [20]:
gf.head(2)


,fill-opacity,fill,roi_percent_coverage,data_epsg,geometry_epsg,datacube_exist,zarr_url,geometry,zarr_s3,tile_id,path_exists
309,0.752848,red,24.715192,EPSG:32642,"{'type': 'Polygon', 'coordinates': [[[500000, ...",1,http://its-live-data.s3.amazonaws.com/datacube...,"POLYGON ((69.00000 34.34130, 70.08714 34.33648...",s3://its-live-data/datacubes/v02/N30E060/ITS_L...,309,True
310,0.243148,red,75.685160,EPSG:32642,"{'type': 'Polygon', 'coordinates': [[[500000, ...",1,http://its-live-data.s3.amazonaws.com/datacube...,"POLYGON ((69.00000 35.24308, 70.09904 35.23809...",s3://its-live-data/datacubes/v02/N30E060/ITS_L...,310,True


In [21]:
gf['path_exists'] = gf.path_exists.astype(str)
gf.explore(column = 'path_exists',
                    cmap = 'Set1',
                    #tiles = 'https://glaceirflow.nyc3.digitaloceanspaces.com/webmaps/vel_map/{z}/{x}/{y}.png',
                    tiles = 'OpenStreetMap',
                    attr = 'ITS_LIVE Velocity Mosaic')

/Users/emarshall/miniconda3/envs/mynewbook/lib/python3.10/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [22]:
import xarray as xr

In [23]:
#Select Zarr s3 url by index
# 225 (75.61, 35.96)
# 226 (75.53, 36.17)

TILEID = 226
LON, LAT = (75.53, 36.17)

s3obj = gf.loc[TILEID].zarr_s3
s3obj

KeyError: 226

In [ ]:
%%time

ds = xr.open_dataset(s3obj, 
                    storage_options = {'anon':True},
                    chunks = 'auto',
                    engine = 'zarr')

In [ ]:
ds.vx

In [ ]:
ds.vx.encoding

In [ ]:
import hvplot.xarray

In [ ]:
import pyproj
UTM = pyproj.CRS.from_epsg(ds.attrs['projection'])
LONLAT = pyproj.CRS.from_epsg(4326)
proj = pyproj.Transformer.from_crs(LONLAT, UTM, always_xy=True)

utmX,utmY = proj.transform(LON, LAT)


In [ ]:
%%time 

# Timeseries plots should be fast, but where to select? lots of nans...

ds.v.sel(x=utmX, y=utmY, method='nearest').hvplot.scatter()